<a href="https://colab.research.google.com/github/badzlan/techfest-elims/blob/main/EDA_tipiel_IPB.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Efektivitas Penyaluran Dana Transfer ke Daerah (TKDD): Studi Perbandingan Pagu dan Realisasi TKDD di Provinsi-Provinsi Indonesia Tahun 2023

## Info

### Rumusan Masalah

1. Bagaimana tingkat perbandingan antara pagu dan realisas TKDD di setiap provinsi di Indonesia tahun 2023?

2. Provinsi mana yang memiliki tingkat realisasi tertinggi dan terendah?

3. Apa kemungkinan faktor-faktor yang mempengaruhi tinggi/rendahnya realisasi TKDD?

4. Apakah terdapat hubungan antara realisasi TKDD dan IPM suatu provinsi?

5. Apa saja faktor-faktor yang memengaruhi IPM di tiap provinsi Indonesia pada tahun 2023

### Tugas peserta

1. Melakukan analisis perbandingan pagu TKDD dan realisasi TKDD

2. Menghitung persentase realisasi TKDD dari tiap provinsi dan mengklasifikasikannya menjadi 3 kategori:
  - Di bawah 90%
  - Berada di antara 90-100%
  - Di atas 100%

3. Menganalissi faktor-faktor yang dapat mempengaruhi realisasi TKDD

4. Analisis hubungan realisasi TKDD dengan IPM

5. Memberikan insight dan rekomendasi kebjijakan yang berdasarkan data

6. Tim wajib mencantumkan penjelasan terkait EDA yang dilakukan

### Catatan

1. Tim peserta wajib mencantumkan penjelasan naratif (deskriptif) terhadap setiap langkah EDA yang dilakukan, baik berupa temuan statistik, pola, tren, maupun anomali, untuk mendukung pemahaman terhadap data dan relevansi dengan studi kasus.

2. Dataset yang disediakan belum sepenuhnya bersih. Setiap peserta diwajibkan melakukan tahap data preparation terlebih dahulu sesuai kebutuhan masing-masing.

3. Peserta diperbolehkan melakukan eksplorasi dan visualisasi data tambahan di luar instruksi yang diberikan. Visualisasi dapat dibuat dengan format dan jenis apapun, selama:
    * Relevan dengan studi kasus
    * Mendukung jawaban terhadap rumusan masalah
    * Menghasilkan insight yang bermakna dan dapat ditindaklanjuti

### Penjelasan Kolom

1. Provinsi: Nama wilayah administratif di Indonesia

2. IPM: Indeks Pembangunan Manusia
    - Angka untuk ngukur kualitas hidup penduduk.
    - Gabungan dari pendidikan, kesehatan, dan pendapatan.
    - Makin tinggi, makin bagus kualitas hidup di daerah itu.

3. Pagu TKDD
    - TKDD = Transfer ke Daerah dan Dana Desa, bagian dari APBN (Anggaran Pendapatan dan Belanja Negara)
    - Total dana transfer dari pusat ke daerah (sebelum dipakai)
    - Ini adalah anggaran yang dikasih dari pusat ke provinsi

4. Realisasi TKDD
    - Seberapa banyak dana TKDD ini yang bener-bener udah dipakai
    - Kalau angkanya lebih kecil dibandingkan dengan pagu, maka artinya belum dimanfaatkan secara maksimal

5. Jumlah Penduduk

6. Anggaran APBN per Kapita
    - Total dana dari pemerintah pusat dibagi per orang
    - Buat tau berapa jatah rata-rata tiap orang dari anggaran nasional

7. Persentase Penduduk Miskin
    - Persentase orang yang hidup di bawah garis kemiskinan
    - Miskin di sini dihitung berdasarkan pengeluaran orang tersebut
    - Ketika pengeluaran < batas bawah garis kemiskinan provinsi tersebut = dikategorikan miskin

8. PRDB Atas Dasar Harga Berlaku
    - PRDB = Produk Domestik Regional Bruto
    - ADHB = Atas Dasar Harga Berlaku
    - Total nilai barang & jasa yang dihasilkan provinsi, dihitung pakai harga saat itu (tanpa koreksi inflasi).
    - Ibaratnya: seberapa “kaya” aktivitas ekonomi daerah itu saat ini.

9. PRDB per Kapita HB
    - PDRB dibagi jumlah penduduk → buat lihat rata-rata kontribusi ekonomi per orang.
    - Makin tinggi, artinya rata-rata orang di daerah itu punya “aktivitas ekonomi” yang tinggi.

10. Laju Pertumbuhan PDRB atas dasar konstan 2010
    - Seberapa cepat ekonomi daerah tumbuh dibanding tahun sebelumnya.
    - Pakai harga konstan 2010 (jadi gak dipengaruhi inflasi).
    - Makin tinggi persen pertumbuhannya, berarti ekonomi makin cepat berkembang.

## 1. Data Preparation

### 1.1 Import library and Data

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import re

In [2]:
# baca data dari github
url = "https://raw.githubusercontent.com/badzlan/techfest-elims/refs/heads/main/dataset.csv"
df = pd.read_csv(url)

# menampilkan 5 data pertama
df.head()

,Provinsi,IPM,Pagu TKDD,Realisasi TKDD,Jumlah Penduduk,Anggaran APBN per kapita,Persentase Penduduk Miskin,Produk Domestik Regional Bruto (PDRB) Atas Dasar Harga Berlaku,Produk Domestik Regional Bruto per Kapita HB,Laju Pertumbuhan PDRB atas dasar konstan 2010
0,ACEH,"74,7","Rp 30.821.480.000.000,00","Rp 31.074.560.000.000,00",5.409.200,"5.744.760,78","14,45","227018,2",41.408.000,"4,23"
1,BALI,"78,01","Rp 11.152.110.000.000,00","Rp 11.064.980.000.000,00",4.374.300,"2.529.543,01","4,25","274358,18",62.294.000,"5,71"
2,BANTEN,"75,77","Rp 17.453.380.000.000,00","Rp 17.467.380.000.000,00",12.167.000,"1.435.635,74","6,17","814122,03",66.147.000,"4,81"
3,BENGKULU,"74,3","Rp 10.570.080.000.000,00","Rp 10.559.790.000.000,00",2.059.400,"5.127.605,13","14,04","96583,09",46.300.000,"4,28"
4,DI YOGYAKARTA,"81,09","Rp 10.355.870.000.000,00","Rp 10.241.290.000.000,00",3.712.600,"2.758.522,33","11,04","180696,04",48.360.000,"5,07"


Pada tahap ini, dilakukan proses import beberapa library Python yang umum digunakan dalam analisis data, yaitu:

- `pandas` dan `numpy` untuk memproses dan menganalisis data dalam format tabel dan angka,

- `matplotlib.pyplot` dan `seaborn` untuk visualisasi data,

- `re` untuk pemrosesan teks jika diperlukan.

Dataset kemudian diambil langsung dari repositori GitHub menggunakan `pandas.read_csv()` dan dimuat ke dalam variabel `df`. Untuk memastikan data berhasil dimuat dengan benar, ditampilkan lima baris pertama dari dataset menggunakan `df.head()`.

### 1.2 Cek Missing Values & Duplikasi

In [3]:
# cek missing values
print("Jumlah missing value per kolom:")
print(df.isnull().sum())

# cek duplikasi
print("\nJumlah baris duplikat:", df.duplicated().sum())

Jumlah missing value per kolom:
Provinsi                                                          0
IPM                                                               4
Pagu TKDD                                                         0
Realisasi TKDD                                                    0
Jumlah Penduduk                                                   2
Anggaran APBN per kapita                                          0
Persentase Penduduk Miskin                                        4
Produk Domestik Regional Bruto (PDRB) Atas Dasar Harga Berlaku    0
Produk Domestik Regional Bruto per Kapita HB                      0
Laju Pertumbuhan PDRB atas dasar konstan 2010                     0
dtype: int64

Jumlah baris duplikat: 0


Pada tahap ini dilakukan pengecekan terhadap nilai kosong (missing values) dan duplikasi data.
Hasilnya menunjukkan bahwa terdapat beberapa kolom dengan nilai kosong, yaitu:

- `IPM`: 4 provinsi tidak memiliki data,

- `Jumlah Penduduk`: 2 provinsi tidak memiliki data,

- `Persentase Penduduk Miskin`: 4 provinsi tidak memiliki data.

Sementara itu, tidak ditemukan baris duplikat dalam dataset, sehingga tidak diperlukan proses penghapusan data ganda.



### 1.3 Renaming Columns

In [4]:
# rename kolom-kolom
df.rename(columns={
    'Provinsi': 'provinsi',
    'IPM': 'ipm',
    'Pagu TKDD': 'pagu_tkdd',
    'Realisasi TKDD': 'realisasi_tkdd',
    'Jumlah Penduduk': 'jumlah_penduduk',
    'Anggaran APBN per kapita': 'apbn_per_kapita',
    'Persentase Penduduk Miskin': 'persentase_penduduk_miskin',
    'Produk Domestik Regional Bruto (PDRB) Atas Dasar Harga Berlaku': 'pdrb_adhb',
    'Produk Domestik Regional Bruto per Kapita HB': 'pdrb_per_kapita_hb',
    'Laju Pertumbuhan PDRB atas dasar konstan 2010': 'laju_pdrb_konstan_2010'
}, inplace=True)

df.columns

Index(['provinsi', 'ipm', 'pagu_tkdd', 'realisasi_tkdd', 'jumlah_penduduk',
       'apbn_per_kapita', 'persentase_penduduk_miskin', 'pdrb_adhb',
       'pdrb_per_kapita_hb', 'laju_pdrb_konstan_2010'],
      dtype='object')

Agar lebih mudah digunakan dalam proses analisis data dan visualisasi, seluruh nama kolom diubah menjadi format **huruf kecil** dan menggunakan **underscore** (`snake_case`).
Penyesuaian ini dilakukan agar sesuai dengan konvensi penulisan Python, sehingga mempermudah pemanggilan kolom dalam pemrograman selanjutnya.

Contohnya:

- `Pagu TKDD` menjadi `pagu_tkdd`

- `Persentase Penduduk Miskin` menjadi `persentase_penduduk_miskin`

- `Produk Domestik Regional Bruto (PDRB) Atas Dasar Harga Berlaku` menjadi `pdrb_adhb`

### 1.4 Cek & Pastikan Tipe Data Numerik

In [5]:
# Cek tipe data
print("\nTipe data setiap kolom:")
print(df.dtypes)

replacement_rules = {
    r'Rp ': '',
    r'\.': '',
    r',': '.'
}

clear_cols = ['ipm',
              'pagu_tkdd',
              'realisasi_tkdd',
              'jumlah_penduduk',
              'apbn_per_kapita',
              'persentase_penduduk_miskin',
              'pdrb_adhb',
              'pdrb_per_kapita_hb',
              'laju_pdrb_konstan_2010'
]

def apply_replacements(text, rules):
    # convert non-string values to string, handling NaN
    if not isinstance(text, str):
        return str(text)
    for pattern, replacement in rules.items():
        text = re.sub(pattern, replacement, text)
    return text

for col in clear_cols:
  df[col] = df[col].apply(lambda x: apply_replacements(x, replacement_rules))
  df[col] = pd.to_numeric(df[col], errors='coerce')

print("\nTipe data setelah konversi:")
print(df.dtypes)


Tipe data setiap kolom:
provinsi                      object
ipm                           object
pagu_tkdd                     object
realisasi_tkdd                object
jumlah_penduduk               object
apbn_per_kapita               object
persentase_penduduk_miskin    object
pdrb_adhb                     object
pdrb_per_kapita_hb            object
laju_pdrb_konstan_2010        object
dtype: object

Tipe data setelah konversi:
provinsi                       object
ipm                           float64
pagu_tkdd                     float64
realisasi_tkdd                float64
jumlah_penduduk               float64
apbn_per_kapita               float64
persentase_penduduk_miskin    float64
pdrb_adhb                     float64
pdrb_per_kapita_hb              int64
laju_pdrb_konstan_2010        float64
dtype: object


Pada tahap ini dilakukan pengecekan dan pembersihan tipe data untuk memastikan semua kolom numerik memiliki tipe data yang sesuai (float64 atau int64).
Awalnya, sebagian besar kolom numerik masih terbaca sebagai object karena mengandung format angka khas Indonesia seperti:

- Simbol `"Rp "`,

- Titik `.` sebagai pemisah ribuan,

Koma `,` sebagai desimal.

Untuk itu, dilakukan pembersihan menggunakan regex (regular expression) dengan aturan penggantian berikut:

- Hapus `"Rp "` dan titik,

- Ganti koma `,` dengan titik `.` agar sesuai dengan format desimal Python.

Setelah dibersihkan, seluruh kolom dikonversi ke tipe numerik menggunakan `pd.to_numeric()`, sehingga proses analisis selanjutnya dapat dilakukan tanpa error.

Hasil akhir menunjukkan bahwa semua kolom numerik berhasil diubah ke tipe data numerik (`float64`/`int64`) dan siap dianalisis lebih lanjut.

### 1.5 Handle Missing Values

In [6]:
# --- list provinsi di sulawesi ---
sulawesi_provinces = [
    "SULAWESI UTARA", "SULAWESI TENGAH", "SULAWESI SELATAN",
    "SULAWESI TENGGARA", "SULAWESI BARAT", "GORONTALO"
]

# --- kolom yang akan diimputasi ---
target_columns = [
    'pagu_tkdd', 'realisasi_tkdd', 'apbn_per_kapita',
]

# --- imputasi hanya untuk provinsi sulawesi ---
for col in target_columns:
    mean_val = df[df['provinsi'].isin(sulawesi_provinces)][col].mean()
    df.loc[df['provinsi'] == 'SULAWESI UTARA', col] = mean_val

Dilakukan imputasi nilai yang hilang pada provinsi **Sulawesi Utara** untuk kolom `pagu_tkdd`, `realisasi_tkdd`, dan `apbn_per_kapita`.
Nilai yang digunakan adalah **rata-rata (mean)** dari provinsi-provinsi lain di Pulau Sulawesi agar tetap relevan secara geografis dan fiskal.



In [7]:
# ipm papua = median nasional
# jumlah penduduk dan apbn = mean regional
# persentase penduduk miskin = mean regional

papua_regional = ['PAPUA BARAT DAYA', 'PAPUA PEGUNUNGAN', 'PAPUA SELATAN', 'PAPUA TENGAH', 'PAPUA', 'PAPUA BARAT']

# handle ipm
papua_ipm = ['PAPUA BARAT DAYA', 'PAPUA PEGUNUNGAN', 'PAPUA SELATAN', 'PAPUA TENGAH']

for prov in papua_ipm :
  df.loc[df['provinsi'] == prov, 'ipm'] = df['ipm'].median()

# handle apbn, jumlah penduduk, persen penduduk miskin
papua_penduduk_apbn = ['PAPUA PEGUNUNGAN', 'PAPUA SELATAN']
col_papua_penduduk_apbn = ['jumlah_penduduk', 'apbn_per_kapita']

for prov in papua_penduduk_apbn :
  for col in col_papua_penduduk_apbn :
    df.loc[df['provinsi'] == prov, col] = df[df['provinsi'].isin(papua_regional)][col].mean()

# handle persentase penduduk miskin
papua_persentase_miskin = ['PAPUA BARAT DAYA', 'PAPUA PEGUNUNGAN', 'PAPUA SELATAN', 'PAPUA TENGAH']

for prov in papua_persentase_miskin :
  df.loc[df['provinsi'] == prov, 'persentase_penduduk_miskin'] = df[df['provinsi'].isin(papua_regional)]['persentase_penduduk_miskin'].mean()

- `ipm` untuk 4 provinsi pemekaran di Papua diisi dengan **median nasional**, karena indikator IPM relatif stabil dan distribusinya tidak terlalu ekstrem.

- `jumlah_penduduk` dan `apbn_per_kapita` untuk Papua Pegunungan dan Papua Selatan diisi dengan **mean dari seluruh provinsi di Papua** agar lebih kontekstual.

- `persentase_penduduk_miskin` pada 4 provinsi pemekaran juga diisi menggunakan **rata-rata (mean)** dari provinsi-provinsi Papua lainnya karena indikator ini sangat dipengaruhi kondisi wilayah setempat.

### 1.6 Tambahkan Kolom Rasio Realisasi TKDD

In [8]:
df['rasio_realisasi'] = (df['realisasi_tkdd'] / df['pagu_tkdd']) * 100

Untuk mengukur efektivitas penyerapan anggaran, ditambahkan kolom baru bernama `rasio_realisasi` yang menunjukkan **persentase realisasi TKDD terhadap pagu TKDD**.
Nilai ini dihitung dengan rumus:

```
rasio_realisasi = (realisasi_tkdd / pagu_tkdd) * 100
```

Kolom ini akan digunakan pada analisis selanjutnya untuk klasifikasi dan perbandingan tingkat efektivitas penyaluran dana antar provinsi.



### 1.7 Klasifikasikan Rasio ke dalam 3 Kategori

In [9]:
def klasifikasi_realisasi(rasio):
    if rasio < 90:
        return '< 90'
    elif 90 <= rasio <= 100:
        return '90 - 100'
    elif rasio > 100:
        return '> 100'
    else :
        return 'Unknown'

df['kategori_realisasi_%'] = df['rasio_realisasi'].apply(klasifikasi_realisasi)
df

,provinsi,ipm,pagu_tkdd,realisasi_tkdd,jumlah_penduduk,apbn_per_kapita,persentase_penduduk_miskin,pdrb_adhb,pdrb_per_kapita_hb,laju_pdrb_konstan_2010,rasio_realisasi,kategori_realisasi_%
0,ACEH,74.70,3.082148e+13,3.107456e+13,5.409200e+06,5.744761e+06,14.45,227018.20,41408000,4.23,100.821116,>100
1,BALI,78.01,1.115211e+13,1.106498e+13,4.374300e+06,2.529543e+06,4.25,274358.18,62294000,5.71,99.218713,90-100
2,BANTEN,75.77,1.745338e+13,1.746738e+13,1.216700e+07,1.435636e+06,6.17,814122.03,66147000,4.81,100.080214,>100
3,BENGKULU,74.30,1.057008e+13,1.055979e+13,2.059400e+06,5.127605e+06,14.04,96583.09,46300000,4.28,99.902650,90-100
4,DI YOGYAKARTA,81.09,1.035587e+13,1.024129e+13,3.712600e+06,2.758522e+06,11.04,180696.04,48360000,5.07,98.893574,90-100
5,DKI JAKARTA,83.55,2.469499e+13,2.508018e+13,1.064000e+07,2.357160e+06,4.44,3443026.23,322619000,4.96,101.559790,>100
6,GORONTALO,71.25,6.172420e+12,6.155410e+12,1.198400e+06,5.136357e+06,15.15,51366.74,42341000,4.50,99.724419,90-100
7,JAMBI,73.73,1.523207e+13,1.535267e+13,3.633200e+06,4.225661e+06,7.58,293780.01,79850000,4.67,100.791751,>100
8,JAWA BARAT,74.24,7.132176e+13,7.103169e+13,4.930680e+07,1.440606e+06,7.62,2625225.72,52652000,5.00,99.593294,90-100
9,JAWA TENGAH,73.39,6.712969e+13,6.677206e+13,3.718040e+08,1.795894e+05,10.77,1695621.57,45167000,4.97,99.467255,90-100


Setelah menghitung persentase realisasi TKDD, dilakukan klasifikasi nilai rasio_realisasi ke dalam tiga kategori:

- `< 90` untuk provinsi dengan tingkat realisasi di bawah 90% (kurang efektif),

- `90 - 100` untuk provinsi dengan realisasi sesuai target (cukup efektif),

- `> 100` untuk provinsi dengan realisasi melebihi pagu (indikasi penambahan anggaran atau efisiensi belanja).

Kategori ini disimpan dalam kolom baru bernama `kategori_realisasi_%`, dan akan digunakan dalam analisis perbandingan serta visualisasi berikutnya.